## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-16-03-55-41 +0000,bbc,"Labubu mania to drive up profit 350%, Pop Mart...",https://www.bbc.com/news/articles/cvg85e0v5vyo
1,2025-07-16-02-55-00 +0000,wsj,Singapore Dollar Consolidates; Abating Fed Rat...,https://www.wsj.com/economy/central-banking/as...
2,2025-07-16-02-04-10 +0000,bbc,Trump claims tariff deal with Indonesia,https://www.bbc.com/news/articles/cwyq0ln6z6lo
3,2025-07-16-01-31-32 +0000,nyt,Israel Stages Rare Attack on Syrian Government...,https://www.nytimes.com/2025/07/15/world/middl...
4,2025-07-16-01-06-47 +0000,bbc,Trump launches probe into Brazil's 'unfair' tr...,https://www.bbc.com/news/articles/cj0mg2j7z04o


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2325/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
17,trump,15
3,up,6
61,tariffs,4
114,inflation,4
171,canada,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
8,2025-07-16-00-00-00 +0000,wsj,"President Trump wants tariffs, the higher the ...",https://www.wsj.com/economy/trade/forget-taco-...,35
5,2025-07-16-01-00-00 +0000,wsj,The impact of President Trump’s whirlwind six ...,https://www.wsj.com/economy/trump-effect-start...,32
19,2025-07-15-19-03-00 +0000,wsj,JPMorgan CEO Jamie Dimon sounded Wall Street’s...,https://www.wsj.com/economy/central-banking/di...,32
37,2025-07-15-10-42-38 +0000,bbc,How Trump woke me up for surprise interview -...,https://www.bbc.com/news/articles/c5yg7eg8w98o,27
39,2025-07-15-10-01-45 +0000,bbc,Trump shrugs off giving Parliament speech when...,https://www.bbc.com/news/articles/cx2g1xexe7qo,27


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
8,35,2025-07-16-00-00-00 +0000,wsj,"President Trump wants tariffs, the higher the ...",https://www.wsj.com/economy/trade/forget-taco-...
15,24,2025-07-15-20-41-00 +0000,wsj,"Inflation picked up in June, a potential sign ...",https://www.wsj.com/economy/inflation-hit-2-7-...
28,20,2025-07-15-14-57-21 +0000,bbc,Two dead after flash flooding in New Jersey an...,https://www.bbc.com/news/articles/c0j42xy47q7o
19,17,2025-07-15-19-03-00 +0000,wsj,JPMorgan CEO Jamie Dimon sounded Wall Street’s...,https://www.wsj.com/economy/central-banking/di...
21,15,2025-07-15-18-27-07 +0000,nyt,Canada Wildfire Smoke Triggers Air Quality Ale...,https://www.nytimes.com/2025/07/14/world/canad...
6,13,2025-07-16-00-23-39 +0000,bbc,Thousands of Afghans were moved to UK in secre...,https://www.bbc.com/news/articles/cvg8zy78787o
42,12,2025-07-15-09-38-38 +0000,bbc,'Not our war' - Trump's Nato weapons deal for ...,https://www.bbc.com/news/articles/c14e2ydv4d6o
16,11,2025-07-15-20-17-00 +0000,wsj,Leading Economies Must Work Together to Fix Im...,https://www.wsj.com/economy/global/leading-eco...
11,10,2025-07-15-22-14-56 +0000,bbc,Adolescence star Owen Cooper becomes Emmys' yo...,https://www.bbc.com/news/articles/cjelpe2y3plo
9,10,2025-07-15-23-48-34 +0000,nyt,Huckabee Calls Death of Palestinian-American i...,https://www.nytimes.com/2025/07/15/world/middl...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
